In [3]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [4]:
from src.connection_db import get_db, get_collection

db = get_db()

# Selecionar o banco de dados
# Selecionar as coleções
colecao1 = get_collection("google_maps_com_nlp")
colecao2 = get_collection("google_play_com_nlp")
colecao3 = get_collection("reclame_aqui_com_nlp")

# Função para converter coleção em DataFrame com tag de origem
def colecao_para_df(colecao):
    dados = list(colecao.find())  # Busca todos os documentos
    df = pd.DataFrame(dados) 
    return df

# Converter cada coleção em DataFrame
df_google_maps_nlp = colecao_para_df(colecao1)
df_google_play_nlp = colecao_para_df(colecao2)
df_reclame_aqui_nlp = colecao_para_df(colecao3)

In [136]:
df_google_maps_nlp.columns


Index(['_id', 'nome', 'nota', 'comentario', 'endereco', 'sentimento',
       'score_sentimento', 'comentario_limpo', 'bairro', 'origem',
       'sentimento_comentario'],
      dtype='object')

In [137]:
df_google_play_nlp.columns

Index(['_id', 'reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'sentimento', 'origem',
       'sentimento_comentario'],
      dtype='object')

In [ ]:
df_reclame_aqui_nlp.columns






In [138]:
def normalizar_df(df, origem, rename_map):
    df = df.copy()
    
    # Renomeia colunas
    df = df.rename(columns=rename_map)

    # Adiciona a origem
    df["origem"] = origem

    # Garante que todas as colunas principais existam
    colunas_finais = [
        "comentario", "data", "origem","sentimento_nlp", 
        "_id", "nome_usuario"
    ]

    for col in colunas_finais:
        if col not in df.columns:
            df[col] = None


    return df[colunas_finais]

In [139]:
rename_ra = {
    "texto": "comentario",
    "sentimento": "sentimento_regra",
    "sentimento_comentario": "sentimento_nlp",
    "titulo": "titulo",
    "data": "data",
    "id": "id_review",
}
rename_play = {
    "content": "comentario",
    "score": "score",
    "sentimento": "sentimento_regra",
    "sentimento_comentario": "sentimento_nlp",
    "userName": "nome_usuario",
    "at": "data",
    "reviewId": "id_review",
    "reviewCreatedVersion": "versao",
    "replyContent": "resposta",
}
rename_maps = {
    "comentario": "comentario",
    "nota": "score",
    "sentimento": "sentimento_regra",
    "sentimento_comentario": "sentimento_nlp",
    "nome": "nome_usuario",
    "endereco": "endereco",
    "bairro": "bairro",
}

df_maps_norm = normalizar_df(df_google_maps_nlp, "google_maps", rename_maps)
df_play_norm = normalizar_df(df_google_play_nlp, "google_play", rename_play)
df_ra_norm = normalizar_df(df_reclame_aqui_nlp, "reclame_aqui", rename_ra)

In [140]:
df_total = pd.concat([df_maps_norm, df_play_norm, df_ra_norm], ignore_index=True)


In [141]:
df_total

,comentario,data,origem,sentimento_nlp,_id,nome_usuario
0,Passei pela loja e fiz um pedido pessoalmente ...,None,google_maps,muito negativo,6903dc7672251ed20a826828,Juliana Linhares
1,"Pedi um delivery, não tinha como vir pior, dur...",None,google_maps,muito negativo,6903dc7672251ed20a826829,Ana Beatriz Barducco
2,Fui super bem atendido pelos profissionais Ing...,None,google_maps,muito positivo,6903dc7672251ed20a82682a,Claudio Lessa
3,"Pedi pela primeira vez pelo delivery, e tudo c...",None,google_maps,negativo,6903dc7672251ed20a82682b,Manuela Scarpelini
4,"Comprei uma pizza de 12 pedaços no ifood, me e...",None,google_maps,muito negativo,6903dc7672251ed20a82682c,Ronaldo murakami
...,...,...,...,...,...,...
3230,Fizemos uma compra de uma pizza no dia 21/10 n...,2025-10-27 13:58:00,reclame_aqui,muito negativo,690408bafe99db6147eb1204,None
3231,Pizza hut do shopping da Bahia precisa de uma ...,2025-10-26 21:13:00,reclame_aqui,muito negativo,690408bafe99db6147eb120e,None
3232,"Fiz um pedido, paguei por via pix, eles inform...",2025-10-26 19:38:00,reclame_aqui,muito negativo,690408bafe99db6147eb121f,None
3233,Fiz um pedido na Pizza Hut pelo RAPPI. O pedid...,2025-10-26 15:16:00,reclame_aqui,muito negativo,690408bafe99db6147eb1220,None


In [ ]:
#dividir dataframe em linhas com datas e linhas sem data
df_com_data = df_total[df_total["data"].notna()].copy()
df_sem_data = df_total[df_total["data"].isna()].copy()

print("Linhas com data:", len(df_com_data))
print("Linhas sem data:", len(df_sem_data))


Linhas com data: 2947
Linhas sem data: 288
3235


In [145]:
def kpi_volume(df):
     return { "total_avaliacoes": len(df),
              "total_por_origem": df["origem"].value_counts(),
              "total_por_sentimento": df["sentimento_nlp"].value_counts(dropna=True) }

def kpi_sentimento(df):

    # garantir que a data seja datetime
    if df["data"].dtype != "datetime64[ns]":
        df["data"] = pd.to_datetime(df["data"], errors="coerce")

    # converter sentimento NLP para escala 1–5
    mapa = {
        "muito positivo": 5,
        "positivo": 4,
        "neutro": 3,
        "negativo": 2,
        "muito negativo": 1
    }

    df["sent_numerico"] = df["sentimento_nlp"].map(mapa).fillna(3)  # neutro como fallback

    kpis = {
        "percentual_sentimentos": df["sentimento_nlp"].value_counts(normalize=True) * 100,
        "sentimento_medio_geral": df["sent_numerico"].mean(),
        "sentimento_medio_por_origem": df.groupby("origem")["sent_numerico"].mean(),
        "sentimento_medio_por_periodo": df.groupby(df["data"].dt.to_period("M"))["sent_numerico"].mean()
    }
    if df["data"].notna().sum() > 0:
        df_validas = df[df["data"].notna()]
        
        kpis["sentimento_medio_por_periodo"] = (
            df_validas.groupby(df_validas["data"].dt.to_period("M"))["sent_numerico"].mean()
        )
        
        kpis["volume_por_periodo"] = (
            df_validas.groupby(df_validas["data"].dt.to_period("M"))["_id"].count()
        )

    return kpis

In [147]:
kpi_vol = kpi_volume(df_total)

# Tabela 1 – Volume por origem 
tabela_volume_origem = kpi_vol["total_por_origem"].reset_index()
tabela_volume_origem.columns = ["origem", "quantidade"]

# Tabela 2 – Volume por sentimento
tabela_volume_sentimento = kpi_vol["total_por_sentimento"].reset_index()
tabela_volume_sentimento.columns = ["sentimento", "quantidade"]

# Sentimentos
kpi_sent = kpi_sentimento(df_total)

# Percentual de sentimentos 
tabela_percentual_sentimento = kpi_sent["percentual_sentimentos"].reset_index()
tabela_percentual_sentimento.columns = ["sentimento", "percentual"]

#sent medio por origem
tabela_sentimento_origem = kpi_sent["sentimento_medio_por_origem"].reset_index()
tabela_sentimento_origem.columns = ["origem", "sentimento_medio"]

if "sentimento_medio_por_periodo" in kpi_sent:
    tabela_sentimento_periodo = kpi_sent["sentimento_medio_por_periodo"].reset_index()
    tabela_sentimento_periodo.columns = ["periodo", "sentimento_medio"]

if "volume_por_periodo" in kpi_sent:
    tabela_volume_periodo = kpi_sent["volume_por_periodo"].reset_index()
    tabela_volume_periodo.columns = ["periodo", "quantidade"]
